# ITBA - Cloud Data Engineering
## Trabajo Práctico Final Foundations 
### 👨🏽‍💻 Flavio Lanfranco - Versión 20210808

El [dataset seleccionado](https://www.kaggle.com/olistbr/brazilian-ecommerce) presenta datos públicos de comercio electrónico pertenecientes al sitio brasileño [Olist Store](https://olist.com).

#### 📌 Acerca del dataset 

El dataset contiene información de aproximadamente 100k pedidos provenientes de los distintos canales de venta que gestiona la plataforma Olist Store. Los mismos se corresponden al periodo que va desde el año 2016 al 2018. El dataset permite ver un pedido a partir de múltiples dimensiones: desde su estado, precio, pago y desempeño del flete, hasta la ubicación del cliente, los atributos del producto y finalmente las reseñas escritas por los clientes.

#### 📌 Contexto 

Olist Store es una plataforma que conecta pequeñas empresas de todo Brasil con diferentes canales de ventas. Esos comerciantes pueden vender sus productos a través de Olist Store y enviarlos directamente a los clientes mediante los socios logísticos de Olist. Después de que un cliente compra el producto en Olist Store, un vendedor recibe una notificación para cumplir con ese pedido. Una vez que el cliente recibe el producto, o vence la fecha estimada de entrega, el cliente recibe una encuesta de satisfacción por correo electrónico donde puede dar una nota por la experiencia de compra y anotar algunos comentarios. 

#### 🔍 Análisis 

Bajo la premisa de que el comercio electrónico está creciendo 📈 continuamente de manera global 🌎, resulta interesante poder evaluar si en 🇧🇷 también se experimenta dicha tendencia, al menos para el periodo compartido en el dataset. 

A continuación se listan algunas preguntas a responder ejecutando consultas sobre dicho dataset:

##### En referencia a la cantidad de pedidos:
1. ¿Es posible visualizar 📈 dicha tendencia? 
2. ¿Cómo se distribuyen 📍 los compradores en Brasil?

##### Respecto a las ventas:

3. Analizando el monto de venta 💵  ¿cómo se distribuyen las mismas a través del tiempo?
4. ¿En qué meses calendario 🗓 el monto de venta es mayor?
5. ¿Y en qué días 📆 de la semana?
6. ¿Cuáles son las 20 categorías de productos 🛒 con mayor monto de venta generado 💰?

##### Finalmente, en referencia a los pagos:

7. ¿Cuáles son los medios de pagos 💳 que prefieren los compradores brasileños?


#### 📢 Antes de empezar:

A continuación se muestra el código desarrollado en python 🐍 correspondiente a la ejecución de consultas SQL sobre la base de datos de Olist, junto con la explotación analítica del resultado de dichas consultas que buscan responder los requerimientos planeteados. 

La idea 💡 es que de manera interactiva puedan ir ejecutando (⌨️ ctrl+enter) cada una de las celdas de código presentadas, para luego poder visualizar el análisis 📊 resultante con el objeto de contrastar 🤔 con las respuestas comentadas.

In [ ]:
# Data manipulation & processing libraries imports
from sqlalchemy import create_engine
from sqlalchemy.pool import NullPool
import pandas as pd

# Visualization libraries imports
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
# Open connection to Olist database
engine = create_engine('postgresql://username:secret@db:5432/database', connect_args={'options': '-csearch_path={}'.format('olistdb')}, poolclass=NullPool)
conn = engine.connect()

##### En referencia a la cantidad de pedidos:
1. ¿Es posible visualizar 📈 dicha tendencia? 


In [ ]:
q1 = (
'''
    SELECT  
        to_char(order_approved_at,'Mon YYYY') as date,
        to_char(order_approved_at,'YYYYMM')::integer as date_ord,
        COUNT(order_id) AS order_qty 
    FROM olistdb.orders
    GROUP BY 
         date,
         date_ord
    ORDER BY 
         date_ord asc
    ;       
        '''
      )
r1 = pd.read_sql(q1, con=conn)
fig1 = go.Figure(
    data=[go.Bar(x=r1['date'], y=r1['order_qty'])],
    layout_title_text="Evolución de la cantidad de órdenes por periodo."
)
fig1.show()

##### Respuesta:

Es posible visualizar una tendencia creciente en referencia a la cantidad de órdenes de compra realizadas en Olist Store. Con lo cual podríamos pensar que también en Brasil se experimenta un crecimiento del comercio electrónico (considerando a Olist Store como representativo del mercado electrónico brasileño para el periodo compartido en el dataset). 

2. ¿Cómo se distribuyen 📍 los compradores en Brasil?

In [ ]:
q2 = '''
        SELECT 
            customer_state as state,
            COUNT(customer_unique_id) as number_customers
        FROM 
            olistdb.order_customer
        GROUP BY 
            state
        ORDER BY 
            number_customers desc
        ;
    '''
r2 = pd.read_sql_query(q2, con=engine)
fig2 = go.Figure(
    data=[go.Bar(x=r2['state'], y=r2['number_customers'])],
    layout_title_text="Análisis del número de clientes por estado."
)
fig2.show()

##### Respuesta:

En su mayoría los compradores se concentran en los estados de São Paulo, Río de Janeiro, y Minas Gerais. Que a su vez son los estados con mayor población en Brasil.

##### Respecto a las ventas:

3. Analizando el monto de venta 💵  ¿cómo se distribuyen las mismas a través del tiempo?


In [ ]:
q3 = (
'''
      SELECT 
          to_char(order_approved_at,'Mon YYYY') as date,
          to_char(order_approved_at,'YYYYMM')::integer as date_ord,
          SUM(b.price) as sales 
      FROM olistdb.orders as a 
      INNER JOIN olistdb.order_items as b 
      ON a.order_id = b.order_id 
      GROUP BY 
          date,
          date_ord
      ORDER BY 
          date_ord asc
    ;       
        '''
      )
r3 = pd.read_sql(q3, con=conn)
fig3 = px.line(r3, x=r3['date'], y=r3['sales'], title='Evolución del monto de venta por periodo.')
fig3.show()

##### Respuesta:
Las ventas también acompañan la tendencia creciente de ordenes de compra.

4. ¿En qué meses calendario 🗓 el monto de venta es mayor?


In [ ]:
q4 = '''
      SELECT 
          to_char(order_approved_at,'Mon') as date,
          to_char(order_approved_at,'MM')::integer as date_ord,
          SUM(b.price) as sales 
      FROM olistdb.orders as a 
      INNER JOIN olistdb.order_items as b 
      ON a.order_id = b.order_id 
      GROUP BY 
          date,
          date_ord
      ORDER BY 
          date_ord asc
        ;
    '''
r4 = pd.read_sql_query(q4, con=engine)
fig4 = go.Figure(
    data=[go.Bar(x=r4['date'], y=r4['sales'])],
    layout_title_text="Análisis del monto de venta por mes calendario."
)
fig4.show()

##### Respuesta: 
Podemos ver que en los meses de marzo a agosto se producen el mayor volumen de ventas. En cambio, para los meses de septiembre a diciembre se corresponden al periodo con menor volumen de ventas.

5. ¿Y en qué días 📆 de la semana?


In [ ]:
q5 = '''
      SELECT 
          to_char(order_approved_at,'Day') as date,
          to_char(order_approved_at,'D')::integer as date_ord,
          SUM(b.price) as sales 
      FROM olistdb.orders as a 
      INNER JOIN olistdb.order_items as b 
      ON a.order_id = b.order_id 
      GROUP BY 
          date,
          date_ord
      ORDER BY 
          date_ord asc
      ;
    '''
r5 = pd.read_sql_query(q5, con=engine)
fig5 = go.Figure(
    data=[go.Bar(x=r5['date'], y=r5['sales'])],
    layout_title_text="Análisis del monto de ventas por día de la semana."
)
fig5.show()

##### Respuesta:
El día de la semana en el que se produce el mayor volumen de ventas es el martes, luego la tendencia es decreciente hasta llegar al domingo donde se produce el menor volumen de ventas.

6. ¿Cuáles son las 20 categorías de productos 🛒 con mayor monto de venta generado 💰?

In [ ]:
q6 = '''
      SELECT
          b.product_category_name AS product,
          SUM(a.price) AS sales
      FROM 
          olistdb.order_items AS a 
          INNER JOIN olistdb.products AS b 
          ON a.product_id = b.product_id 
      GROUP BY 
         product
      order by
         sales desc
     limit 20
    ; 
      ;
    '''
r6 = pd.read_sql_query(q6, con=engine)
fig6 = go.Figure(
    data=[go.Bar(x=r6['product'], y=r6['sales'])],
    layout_title_text="Ranking de categorías de productos por monto de ventas."
)
fig6.show()

##### Respuesta:
Entre las principales categorías de productos que concentran el mayor volumen de ventas podemos nombrar las relacionadas a artículos de belleza (beleza_saude), relojes y presentes (relogios_presentes), artículos de baño y habitación (cama_mesa_banho), deportes y ocio (esporte_lazer) y accesorios informáticos (informatica_acessorios).

##### Finalmente, en referencia a los pagos:

7. ¿Cuáles son los medios de pagos 💳 que prefieren los compradores brasileños?

In [ ]:
q7 = '''
    SELECT 
        payment_type, 
        COUNT(*) as number_payments
    FROM 
        olistdb.order_payments
    GROUP BY 
        payment_type
    ;
    '''
r7 = pd.read_sql_query(q7, con=engine)
fig7 = px.pie(r7, values=r7['number_payments'], names=r7['payment_type'], title='Distribución del número de pagos por tipo.')
fig7.show()

##### Respuesta:
En su gran mayoría el comprador brasileño prefiere la tarjeta de crédito como su principal medio de pago, luego el boleto en una menor medida. 

In [ ]:
# Closing connection to Olist database
conn.close()



### Conclusiones generales respecto al análisis presentado

    📌 Olist Store presenta una tendencia creciente de ventas.
    📌 Los últimos 4 meses del año parecen ser los de menor performance de ventas.
    📌 Los principales centros urbanos parecen tener una gran influencia en las ventas.
    📌 El pago con tarjeta de crédito lidera las ventas on-line.

En base a estos análisis introductorios presentados podría proponerse al departamento de marketing de Olist Store estudiar en detalle la realización de campañas de marketing encofadas a maximizar las ventas para los principales centros urbanos y categorías anteriormente nombradas, para los días martes, y con foco en promociones para tarjetas de crédito. 